1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
    토큰수 초과로 답변을 생성하지 못할수 있고
    문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [42]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter)

In [43]:
len(document_list)

187

In [44]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')



In [51]:
from langchain_community.vectorstores import Chroma

# chroma는 인메모리이기에 디렉토리에 저장을 하자
database = Chroma.from_documents(document_list, embedding=embedding, collection_name="choroma-tax", persist_directory="./chroma")


In [52]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query)

In [53]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')


In [54]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrived_docs}

Question: {query}
"""

In [55]:
ai_message = llm.invoke(prompt)

In [56]:
ai_message.content

'직장인의 소득세를 계산하기 위해서는 여러 가지 요소를 고려해야 합니다. 그 요소에는 소득세율, 각종 공제(기본공제, 특별소득공제 등), 그리고 비과세 소득 등이 포함됩니다. 아래는 기본적인 계산 절차입니다.\n\n1. **과세 표준 계산**:\n   - 총급여에서 근로소득공제 및 기타 공제 항목을 차감하여 과세 표준을 계산합니다.\n   - 예를 들어, 연봉 5천만원인 경우, 근로소득공제를 먼저 계산합니다.\n\n2. **근로소득공제**:\n   - 총 급여가 5천만원인 경우, 대략적인 근로소득공제는 1,470만원입니다. (정확한 금액은 소득에 따라 달라질 수 있으니, 최신 세법 규정을 참조하여 계산해야 합니다.)\n\n3. **과세 표준**:\n   - 5천만원 - 1,470만원 = 3,530만원 (과세 표준)\n\n4. **소득세 계산**:\n   - 과세 표준에 해당하는 세율을 적용하여 소득세를 계산합니다.\n   - 여기서 통상적으로 적용되는 세율 구조는 다음과 같습니다. (정확한 세율은 최신 세법을 참조해야 합니다.)\n     - 1,200만원 이하: 6%\n     - 1,200만원 초과 ~ 4,600만원 이하: 15%\n     - 4,600만원 초과 ~ 8,800만원 이하: 24%\n     - 기타 상위 구간은 별도의 세율 적용\n\n   - 계산을 통해:\n     - 1,200만원까지는 6% 세율 적용: 1,200만원 * 0.06 = 72만원\n     - 나머지 2,330만원 (3,530만원 - 1,200만원)은 15% 세율 적용: 2,330만원 * 0.15 = 349.5만원\n\n5. **기타 공제 및 세액공제**:\n   - 각종 세액공제(예: 근로세액공제, 자녀세액공제 등)를 반영하여 최종 소득세를 계산합니다.\n\n6. **합계**:\n   - 총 세액: 421.5만원에서 기타 공제를 반영한 후의 세금을 산출합니다.\n\n정확한 계산을 위해서는 최신 한국 세법 및 본인의 구체적인 재정 상황(기타 소득, 공제 항목 등)을 고려해야

In [57]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

d:\00-git\LLMS\rag\env\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [58]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])